In [1]:
!pip install langchain openai langchain_openai langchain_core langgraph langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 15.4 MB/s eta 0:00:00


In [ ]:
import os,getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = "open_api_key"

In [3]:
import datetime
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o")

def CurrentDateTime() -> str:
  "This tool provides the current date, time and day of the week in your location"

  dt = datetime.datetime.now()
  return "Datetime is {} and today is {}".format(dt, dt.strftime('%A'))

time_keeper_agent = create_react_agent(
    model=llm,
    tools=[CurrentDateTime],
    name = "time_keeper_agent",
    prompt = "You tell the current date, time and day of week in your location"
)


In [10]:
def ShopOpeningHours() -> str:
  "This tool provides the daily operning hours schedule for shop"
  schedule = """Monday 10:00 to 18:00hrs
                Tuesday 10:00 to 18:00hrs
                Wednesday 10:00 to 18:00hrs
                Thrusday 10:00 to 18:00hrs
                Friday 10:00 to 18:00hrs
                Saturday 10:00 to 18:00hrs
                Sunday 10:00 to 18:00hrs"""
  return schedule

schedule_keeper_agent = create_react_agent(
    model=llm,
    tools=[ShopOpeningHours],
    name = "schedule_keeper_agent",
    prompt = "You tell the shop operationg hours for each day in week"
)

In [11]:
from langgraph_supervisor import create_supervisor

prompt = ("""You are team supervisor to manage time_keeper_agent and scheduled_keeper_agent
Your job is to resond to shop operning hour queries using time_keeper_agent and scheduled_keeper_agent.
To get the current time and date use time_keeper_agent and
for getting shop operating hours use scheduled_keeper_agent
You can use both agents in parallel to get required information""")

workflow = create_supervisor(
    [schedule_keeper_agent, time_keeper_agent],
    model=llm,
    output_mode ="full_history",
    prompt = prompt
)

In [14]:
app = workflow.compile()
result = app.invoke({
    "messages" :[
        {
            "role" : "user",
            "content" : "What are shop's opening hours on weekend? Is the shop open right now"
        }
    ]
})

for m in result['messages']:
    m.pretty_print()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************C8YA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}